In [1]:
import os
import time
import functools
from shutil import copy
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
import hcipy as hc
os.chdir('/Users/asahoo/repos/PASTIS/')


In [2]:
import pastis.util as util
from pastis.config import CONFIG_PASTIS
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC

In [3]:
root_dir = CONFIG_PASTIS.get('local', 'local_data_path')


In [4]:
root_dir

'/Users/asahoo/Desktop/data_repos/laurent_data'

In [5]:
output_dir = CONFIG_PASTIS.get('local', 'output_data_folder')

In [6]:
output_dir

'/Users/asahoo/Desktop/data_repos/laurent_data'

In [11]:
coronagraph_design = 'small'
overall_dir = output_dir + 'luvoirA-'+ coronagraph_design
resDir = os.path.join(overall_dir, 'matrix_numerical')